# 한신대학교 e-비즈니스학과 201646006 이성범 PD학기제 과제

# Linear Regression

회귀란 여러 개의 독립변수와 한 개의 종속 변수간의 상관관계를 모델링하는 기법이다. 머신러닝 회귀 예측의 핵심은 주어진 피쳐와 결정 값 데이터 기반에서 학습을 통해 최적의 회귀 계수를 찾아내는 것이다.

가본적으로 회귀는 독립변수의 개수에 따라 1개이면 단일 회귀로 2개 이상이면 다중 회귀로 나눠지며 회귀 계수의 결합 방식에 따라 선형이라면 선형회귀로 비선형이라면 비선형회귀로 나뉘어 진다.

이번 장에서는 대표적인 회귀 알고리즘인 Linear Regression의 방식에 대해서 공부할 예정이다.

Linear Regression의 회귀계수를 구하는 방식으로는 크게
- 정규방정식
- 경사하강법
- RSS(Residual Sum of Square)를 최소화하는 OLS(Ordinary Least Squares) 방식

3가지로 나뉘어진다.

위 3가지 방식 중 정규방정식을 통해 추정한 회귀계수, sklearn의 Linear Regression이 추정한 회귀게수를 비교해볼 예정이다.


In [ ]:
import numpy as np
from numpy.linalg import inv

np.random.seed(22)

# 다중선형회귀 식 구현
x1 = np.random.randint(1, 15, size = (10,1))
x2 = np.random.randint(800, 1000, size = (10,1))
x3 = np.random.randn(10,1)
x4 = np.random.randint(100, 300, size = (10,1))
X = np.concatenate([x1, x2, x3, x4], axis = 1)

y = np.random.randint(1, 100, size = (10) )

In [ ]:
import pandas as pd

df = pd.DataFrame(X, columns = ['x1', 'x2', 'x3', 'x4'])
df['y'] = y
df

,x1,x2,x3,x4,y
0,6.0,946.0,-0.758910,203.0,60
1,5.0,814.0,0.598314,147.0,94
2,13.0,845.0,1.006748,213.0,66
3,1.0,893.0,-0.760422,229.0,94
4,5.0,967.0,-0.933099,275.0,22
5,7.0,834.0,1.556744,183.0,54
6,7.0,969.0,-1.214495,158.0,10
7,12.0,927.0,1.931360,160.0,85
8,5.0,808.0,1.675647,148.0,18
9,9.0,968.0,0.697013,193.0,53


## 정규방정식

### $\hat{\beta} = (X^{T}X)^{-1}X^{T}y $

정규 방정식을 통한 회귀계수 추정방법은 위와 같은 식으로 이루어져 있으며 반복이 아닌 수학적 수식을 통하여 회귀계수를 추정하기 때문에 독립변수의 개수가 많아질수록 속도가 매우 느려진다는 단점이 존재한다.

In [ ]:
def NormalEquation(X, y):
  # np.ones((len(y), 1)) : 관측치 개수(len(y)) 만큼의 1로 채워진 행렬을 만듬 -> x0 즉 절편을 구하기 위한 새로운 변수임
  # np.concatenate([np.ones((len(y), 1)), X] : x0과 X를 합쳐서 새로운 (9,6)의 X 행렬을 만듬
  designX = np.concatenate([np.ones((len(y), 1)), X], axis=1)
  
  # 정규방정식 구현
  # 1. np.dot(designX.T, designX) : 전치한 X 행렬과 X를 곱함
  # 2. inv(np.dot(designX.T, designX)) : 1번 식의 역행렬을 구함
  # 3. np.dot(inv(np.dot(designX.T, designX)), designX.T) : 역행렬과 전치한 X를 곱함
  # 4. np.dot(np.dot(inv(np.dot(designX.T, designX)), designX.T), y) : 3번 식으로 구한 행렬과 y를 곱함
  beta_hat = np.dot(np.dot(inv(np.dot(designX.T, designX)), designX.T), y)
  
  return beta_hat

In [ ]:
beta_hat = NormalEquation(X, y)
print(beta_hat[0]) # intercept
print(beta_hat[1:]) # 추정된 회귀계수 (intercept제외)

134.4083718352094
[-0.0252083  -0.10965965  4.02658997  0.09541173]


## sklearn의 Linear Regression

- fit_intercept = True / 절편 값을 사용할지 여부 
- normalize = False / 입력 값을 정규화 할건지 여부

LinearRegression은 예측 값과 실제 값의 RSS를 최소화해 OLS 추정 방식으로 구현한 클래스이다. 예측값과 실제 값의 RSS를 최소화할 수 있도록 회귀 계수를 최적화하며, 규제를 적용하지 않은 모델이다. (규제란? 일반적인 선형회귀의 과적합 문제를 해결하기 위해서 회귀 계수에 패널티 값을 적용하는 것)

OLS 기반의 회귀 계수 계산은 입력 피쳐의 독립성에 많은 영향을 받는다. 피처간의 상관관계가 매우 높은 경우 분산이 매우 커져서 오류에 매우 민감해진다. 이러한 현상을 다중공선성 문제라고 한다. 일반적으로 상관관계가 높은 피쳐가 많은 경우 독립적인 중요한 피쳐만 남기고 제거하거나 규제를 적용한다. 또한 매우 많은 피쳐가 다중 공선성 문제를 가지고 있다면 PCA를 통해 차원 축소를 수행하는 것도 고려할 수 있다.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)

print(lr.intercept_) # intercept
print(lr.coef_) # 추정된 회귀계수 (intercept제외)

134.40837183521018
[-0.0252083  -0.10965965  4.02658997  0.09541173]


정규 방정식을 통해서 추정한 회귀계수와 동일하다는 것을 알 수 있다.

# 그 밖의 회귀 모델

## Ridge

- Ridge: Ridge는 선형 회귀에 L2 규제를 추가한 회귀 모델이다. 릿지 회귀는 L2 규제를 적용하는데 L2 규제는 상대적으로 큰 회귀 계수 값의 예측 영향도를 감소시키기 위해서 회귀 계수값을 더 작게 만드는 규제 모델이다.

## Lasso

- Lasso: Lasso는 선형 회귀에 L1 규제를 적용한 방식이다. L2규제가 회귀 계수 값의 크기를 줄이는데 반해, L1 규제는 예측 영향력이 작은 피처의 회귀 계수를 0으로 만들어 회귀 예측 시 피처가 선택되지 않게 하는 것이다. 이러한 특성 때문에 L1 규제는 피처 선택 기능으로도 불린다.

## ElasticNet

- ElasticNet: L2, L1 규제를 함께 결합한 모델이다. 주로 피처가 많은 데이터 세트에 적용되며, L1 규제로 피처의 개수를 줄임과 동시에 L2 규제로 계수 값의 크기를 조정한다.

## CART 기반의 알고리즘

앞으로 배울 RandomForest, LGBM, SVM, XGB 모두 분류뿐 아니라 회귀도 예측이 가능한 모형이며 사실상 회귀 모델을 제외하고는 대부분의 모델이 분류와 회귀를 모두 예측할 수 있다.

# 회귀 평가 지표

- MAE : Mean Absolute Error - 실제 값과 예측 값의 차이를 절대값으로 변환해 평균을 낸 값이다. 0에 가까울 수록 좋다.
- MSE : Mean Squared Error - 실제 값과 예측 값의 차이를 제곱해 평균한 값이다. 0에 가까울 수록 좋다.
- RMSE : Root Mean Squared Error - MSE에 Root를 쒸운 값이다. 0에 가까울 수록 좋다.
- MSLE : Mean Squared Log Error - MSE에 Log를 취한 값이다. 0에 가까울수록 좋다.
- RMSLE : Root Mean Squared Log Error - MSLE에 Root를 쒸운 값이다. 0에 가까울 수록 좋다.
- R^2 : 분산을 기반으로 예측 성능을 평가하는 방식이다. 1에 가까울 수록 좋다.
- ,,,,

등이 존재한다. 

다양한 평가지표가 존재하는 이유는 데이터 마다 예측 값과 실제 값 사이에 패널티의 기준을 다르게 설정하고자 하기 때문이다.

예를 들어 택배업에서 배송까지 하루가 걸린다고 가정했을 때 실제로 배송 시간이 하루보다 더 적게 걸리는 것은 고객 입장에서 큰 상관이 없지만 하루보다 더 걸린다면 고객 입장에서 큰 불만을 가지게 된다. 이렇게 된다면 기업 입장에서 큰 손해를 보게 될 것이다. 

이러한 상황의 경우 실제 값(실제 배송까지 걸린 시간)보다 예측 값(우리가 예측한 배송 시간)을 더 작게 예측했을 때 큰 패널티를 가지게 되는 경우이기 때문에 RMSLE를 평가 기준으로 정해 더 작게 예측하는 값에 패널티를 부여하여 우리의 상황에 맞게 모델의 성능을 평가할 수 있다.

In [ ]:
# MAE 
def MAE(y_true, y_pred):
  return np.mean(np.abs((y_true - y_pred)))

# MSE
def MSE(y_true, y_pred):
  return np.mean(np.square((y_true - y_pred)))

# RMSE
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean(np.square((y_true - y_pred))))

# MSLE
def MSLE(y_true, y_pred):
  return np.mean(np.square((np.log1p(y_true) - np.log1p(y_pred))))

# RMSLE
def RMSLE(y_true, y_pred):
  return np.sqrt(np.mean(np.square((np.log1p(y_true) - np.log1p(y_pred)))))

# R_Sqaure
def R_Sqaure(y_true, y_pred):
  return y_pred.var() / y_true.var()

print(f"MAE      : {MAE(y, y_pred) : 0.4f}")
print(f"MSE      : {MSE(y, y_pred) : 0.4f}")
print(f"RMSE     : {RMSE(y, y_pred) : 0.4f}")
print(f"MSLE     : {MSLE(y, y_pred) : 0.4f}")
print(f"RMSLE    : {RMSLE(y, y_pred) : 0.4f}")
print(f"R_Sqaure : {R_Sqaure(y, y_pred) : 0.4f}")

MAE      :  23.5858
MSE      :  775.0106
RMSE     :  27.8390
MSLE     :  0.4611
RMSLE    :  0.6791
R_Sqaure :  0.0959


현재 나의 상황에 맞게 모델의 성능을 평가하기 위해서 적절한 평가지표를 사용해야 할 것이다.

# 회귀 실습

kaggle 에 있는 중고차 가격 예측 데이터 세트를 가지고 실습을 진행할 예정이다.

# 참고자료
- 파이썬 머신러닝 완벽 가이드, 권철민
- 파이썬 딥러닝 파이토치, 이경택
- 핸즈온 머신러닝, 박해선
- 데이터 사이언스 스쿨, https://datascienceschool.net/
- 데이터 분석 대표 대학생 연합 동아리 Tobigs, http://www.datamarket.kr/
- DeepLearnig.AI Course 1 ~ 5, Andrew Ng, https://www.youtube.com/c/Deeplearningai/playlists
- 모두를 위한 딥러닝 시즌 2, https://deeplearningzerotoall.github.io/season2/
- https://partrita.github.io/posts/regression-error/
- https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-17-%ED%9A%8C%EA%B7%80-%ED%8F%89%EA%B0%80-%EC%A7%80%ED%91%9C